In [1]:
import torchvision
import torch
import torch.nn as nn

import matplotlib.pyplot as plt
import numpy as np

### Writing the code for scanning as per Pavan sir's classes

In [2]:
K = 20
test_image = torch.randn(3, 32, 32)
C, W, H = test_image.shape
D = np.array([C * K * K, 128, 32, 10])
L = D.size

w = [np.random.random(size = (D[d_l], D[d_l + 1])) for d_l in range(L - 1)]
Y = [[np.zeros(shape = (W - K + 1, H - K + 1)) for i in range(d_l)] for d_l in D]
z = [[np.zeros(shape = (W - K + 1, H - K + 1)) for i in range(d_l)] for d_l in D]

In [3]:
img = test_image.numpy()

In [4]:
def activation(h):
    return np.tanh(h)

In [5]:
for x in range(W - K + 1):
    for y in range(H - K + 1):
        for l in range(L):
            for j in range(D[l]):
                if(l == 0):
                    flattened_img = img[:, x:x+K-1, y:y+K-1].flatten()
                    for a in range(len(flattened_img)):
                        Y[0][a][x][y] = flattened_img[a]
                else:
                    for i in range(D[l - 1]):
                        z[l][j][x][y] += w[l - 1][i][j] * Y[l - 1][i][x][y]
                    Y[l][j][x][y] = activation(z[l][j][x][y])

In [6]:
print(len(Y))
print(len(Y[0]))
print(Y[0][0].shape)

4
1200
(13, 13)


### Torch implementation of conv 2d

In [7]:
C_out = 2
C_in = 3
K = 3
stride = 1
W = 32
H = 32
# With square kernels and equal stride
m = nn.Conv2d(C_in, C_out, K, stride=stride, bias = False)
# non-square kernels and unequal stride and with padding
# m = nn.Conv2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2))
# non-square kernels and unequal stride and with padding and dilation
# m = nn.Conv2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2), dilation=(3, 1))

test_input = torch.randn(1, C_in, W, H)
output = m(test_input)

In [8]:
output.shape

torch.Size([1, 2, 30, 30])

In [9]:
test_input.shape

torch.Size([1, 3, 32, 32])

### Testing if the code implemented in Pavan sir's class is the same as conv2d

In [10]:
image = test_input[0].detach().numpy()
C, W, H = image.shape
D = np.array([C_in * K * K, C_out])
L = D.size

w0 = [np.random.random(size = (D[d_l], D[d_l + 1])) for d_l in range(L - 1)]
w = next(iter(m.parameters())).detach().numpy()
Y = np.zeros(shape = (C_out, W - K + 1, H - K + 1))

In [17]:
len(w0), w0[0].shape

(1, (27, 2))

In [18]:
w.shape

(2, 3, 3, 3)

In [11]:
for C_j in range(C_out):
    for x in range(W - K + 1):
        for y in range(H - K + 1):
                for i in range(C_in):
                    Y[C_j][x][y] = np.sum(w[C_j] * image[:, x:x + K, y:y + K])

In [12]:
for weight in m.parameters():
    print(weight.shape)

torch.Size([2, 3, 3, 3])


In [19]:
Y.shape

(2, 30, 30)

In [14]:
output_pytorch = output.detach().numpy()

In [15]:
np.linalg.norm(Y - output_pytorch)

2.4901573357176675e-06